In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext,SparkSession
from pyspark.sql.types import *
from sagemaker_pyspark import IAMRole, classpath_jars
from sagemaker_pyspark.algorithms import KMeansSageMakerEstimator

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/ec2-user/snowflake/spark-snowflake_2.11-2.8.1-spark_2.4.jar,/home/ec2-user/snowflake/snowflake-jdbc-3.12.9.jar pyspark-shell'

In [3]:
conf = (SparkConf()
        #.set("spark.driver.extraClassPath", (":".join(classpath_jars())+":"+":".join(sfc_jars)))
        .setMaster('local')
        .setAppName('local-spark-test'))
sc=SparkContext.getOrCreate(conf=conf)
sc.stop()

sc=SparkContext(conf=conf)
print (sc)

spark = SQLContext(sc)
print (spark)

<SparkContext master=local appName=local-spark-test>


In [20]:
sfOptions = {
  "sfURL" : "https://oq51261.eu-west-1.snowflakecomputing.com/",
  "sfAccount" : "oq51261",
  "sfUser" : "sagemaker",
  "sfPassword" : "IRISPW",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "WEATHER",
  "sfWarehouse" : "SAGEMAKER_WH",
}

In [22]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query", \
"select (V:main.temp_max - 273.15) * 1.8000 + 32.00 as temp_max_far, " +\
"       (V:main.temp_min - 273.15) * 1.8000 + 32.00 as temp_min_far, " +\
"       cast(V:time as timestamp) time, " +\
"       V:city.coord.lat lat, " +\
"       V:city.coord.lon lon " +\
"from snowflake_sample_data.weather.weather_14_total limit 50000000").load()

In [23]:
df.count()

50000000

In [7]:
# convert to pandas dataframe
weather_df = df.toPandas()

KeyboardInterrupt: 